# Probabilidades e Adições

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

Este capítulo apresenta uma nova forma de representar um grau de certeza, **odds***, e uma nova forma do Teorema de Bayes, chamada **Bayes's Rule***.

Bayes's Rule é conveniente se quiser fazer uma actualização Bayesiana em papel ou na sua cabeça.

Também lança luz sobre a importante ideia de **evidência** e como podemos quantificar a força das provas.

A segunda parte do capítulo é sobre "adições", ou seja, quantidades a serem adicionadas, e como podemos calcular as suas distribuições.

Vamos definir funções que calculam a distribuição de somas, diferenças, produtos, e outras operações.

Depois utilizaremos essas distribuições como parte de uma actualização Bayesiana.

## Probabilidades

Uma maneira de representar uma probabilidade é com um número entre 0 e 1, mas essa não é a única maneira.

Se alguma vez apostou num jogo de futebol ou numa corrida de cavalos, provavelmente encontrou outra representação de probabilidade, chamada **odds****.

Pode ter ouvido expressões como "as probabilidades são de três para um", mas pode não saber o que isso significa.

Os **odds a favor** de um evento são a razão da probabilidade

ocorrerá à probabilidade de que não ocorrerá.

A seguinte função faz este cálculo.

In [4]:
def odds(p):
    return p / (1-p)

Por exemplo, se a minha equipa tem 75% de hipóteses de ganhar, as probabilidades a seu favor são de três para um, porque a hipótese de ganhar é três vezes maior do que a hipótese de perder.

In [5]:
odds(0.75)

É possível escrever as probabilidades em forma decimal, mas também é comum

escreva-os como uma proporção de números inteiros.

Assim, "três para um" é por vezes escrito $3:1$.

Quando as probabilidades são baixas, é mais comum reportar o

**odifica contra*** em vez das probabilidades a favor.

Por exemplo, se o meu cavalo tiver 10% de hipóteses de ganhar, as probabilidades a favor são de $1:9$.

In [6]:
odds(0.1)

Mas nesse caso seria mais comum eu dizer que as probabilidades contra são de $9:1$.

In [7]:
odds(0.9)

Dadas as probabilidades a favor, em forma decimal, pode converter-se à probabilidade desta forma:

In [8]:
def prob(o):
    return o / (o+1)

Por exemplo, se as probabilidades forem de $3/2$, a probabilidade correspondente é de $3/5$:

In [9]:
prob(3/2)

Ou se representar probabilidades com um numerador e denominador, pode converter para probabilidade como este:

In [10]:
def prob2(yes, no):
    return yes / (yes + no)

In [11]:
prob2(3, 2)

Probabilidades e probabilidades são diferentes representações do

a mesma informação; dada uma delas, pode calcular a outra.

Mas alguns cálculos são mais fáceis quando trabalhamos com probabilidades, como veremos na secção seguinte, e alguns cálculos são ainda mais fáceis com probabilidades de registo, que veremos mais tarde.

## Regra de Bayes

Até agora, temos trabalhado com o teorema de Bayes na "forma de probabilidade": 

$$P(H|D) = \frac{P(H)~P(D|H)}{P(D)}$$

Escrevendo $\mathrm{odds}(A)$ para probabilidades a favor de $A$, podemos expressar o Teorema de Bayes em "forma de probabilidades":

$$\mathrm{odds}(A|D) = \mathrm{odds}(A)~\frac{P(D|A)}{P(D|B)}$$$

Esta é a regra de Bayes, que diz que as probabilidades posteriores são as probabilidades anteriores vezes o rácio de probabilidade.

Bayes's Rule é conveniente para calcular uma actualização Bayesiana em papel ou na sua cabeça. Por exemplo, voltemos ao problema do cookie:

> Suponha que existem duas tigelas de biscoitos. A tigela 1 contém 30 bolachas de baunilha e 10 bolachas de chocolate. A tigela 2 contém 20 de cada. Suponha agora que escolhe uma das tigelas ao acaso e, sem olhar, seleccione uma bolacha ao acaso. A bolacha é de baunilha. Qual é a probabilidade de ter vindo da Tigela 1?

A probabilidade anterior é de 50%, pelo que as probabilidades anteriores são de 1. O rácio de probabilidade é de $3frac{3}{4}. /frac{1}{2}$, ou $3/2$. Assim, as probabilidades posteriores são de $3/2$, o que corresponde à probabilidade de $3/5$.

In [12]:
prior_odds = 1
likelihood_ratio = (3/4) / (1/2)
post_odds = prior_odds * likelihood_ratio
post_odds

In [13]:
post_prob = prob(post_odds)
post_prob

Se desenharmos outro biscoito e for chocolate, podemos fazer outra actualização:

In [14]:
likelihood_ratio = (1/4) / (1/2)
post_odds *= likelihood_ratio
post_odds

E converter de volta à probabilidade.

In [15]:
post_prob = prob(post_odds)
post_prob

## Oliver's Blood

Iâ™ll usa Bayesâ™s Regra para resolver outro problema de MacKayâ™s

[*Information Theory, Inference, and Learning Algorithms*](https://www.inference.org.uk/mackay/itila/):

> Duas pessoas deixaram vestígios do seu próprio sangue na cena de um crime. Um suspeito, Oliver, é testado e descoberto que tem sangue do tipo â??™â?. Os grupos sanguíneos dos dois traços são encontrados do tipo â??COPY15™â? (um tipo comum na população local, com frequência de 60%) e do tipo â??COPY10™â? (um tipo raro, com frequência de 1%). Será que estes dados [os vestígios encontrados no local] dão provas a favor da proposta de que Oliver era uma das pessoas [que deixou sangue no local]?

Para responder a esta pergunta, precisamos de pensar sobre o que significa para os dados

dar provas a favor de (ou contra) uma hipótese. Intuitivamente, poderíamos dizer que os dados favorecem uma hipótese se a hipótese for mais provável à luz dos dados do que era antes.

No problema do cookie, as probabilidades anteriores são de 1, o que corresponde a 50% de probabilidade. As probabilidades posteriores são de $3/2$, ou probabilidade 60%. Portanto, o cookie de baunilha é uma prova a favor da Taça 1.

A Regra de Bayes fornece uma forma de tornar esta intuição mais precisa. Novamente

$$\mathrm{odds}(A|D) = \mathrm{odds}(A)~\frac{P(D|A)}{P(D|B)}$$$

Dividindo por $\mathrm{odds}(A)$, recebemos: 

$$$frac{\mathrm{mathrm{odds}(A|D)}{\mathrm{mathrm{odds}(A)} = $$frac{P(D|A)}{P(D|B)}$$

O termo à esquerda é a razão entre as probabilidades posterior e anterior. O termo da direita é o rácio da probabilidade, também chamado **Bayes

factor**.

Se o factor Bayes for superior a 1, isso significa que os dados foram

mais provavelmente abaixo de $A$ do que abaixo de $B$. E isso significa que as probabilidades são

maiores, à luz dos dados, do que eram antes.

Se o factor Bayes for inferior a 1, isso significa que os dados eram menos prováveis abaixo de $A$ do que abaixo de $B$, pelo que as probabilidades a favor de $A$ diminuem.

Finalmente, se o factor Bayes for exactamente 1, os dados são igualmente prováveis

sob qualquer das hipóteses, pelo que as probabilidades não mudam.

Vamos aplicar isto ao problema em questão. Se Oliver é uma das pessoas que deixou sangue no local do crime, ele é responsável pela amostra de â??COPY15 â?" 19; nesse caso, a probabilidade dos dados é a probabilidade de um membro aleatório da população ter sangue do tipo â??COPY10 â?" 19, que é 1%.

Se Oliver não deixou sangue no local, temos duas amostras para

conta. 

Se escolhermos duas pessoas aleatórias da população, qual é a hipótese de encontrarmos uma com o tipo â??™â? e outra com o tipo â??˜ABââ???. 

Bem, há duas maneiras de isso acontecer: 

* A primeira pessoa pode ter â??™ e a segunda â??˜ABâ {\i1}COPY19, 

* Ou a primeira pessoa pode ter â˜ABâ™ e a segunda â˜Oâ™.

A probabilidade de qualquer combinação é de $(0,6) (0,01)$, que é de 0,6%, pelo que a probabilidade total é o dobro, ou 1,2%.

Portanto, os dados são um pouco mais prováveis se Oliver for *não* uma das pessoas que deixaram sangue no local.

Podemos utilizar estas probabilidades para calcular o rácio de probabilidade:

In [16]:
like1 = 0.01
like2 = 2 * 0.6 * 0.01

likelihood_ratio = like1 / like2
likelihood_ratio

Como o rácio de probabilidade é inferior a 1, os testes sanguíneos são a prova * contra a hipótese de que Oliver deixou sangue no esquecimento.

Mas é uma prova fraca.  Por exemplo, se as probabilidades anteriores fossem 1 (ou seja, 50% de probabilidade), as probabilidades posteriores seriam 0,83, o que corresponde a uma probabilidade de 45%:

In [17]:
post_odds = 1 * like1 / like2
prob(post_odds)

Portanto, esta prova não "move muito a agulha".

Este exemplo é um pouco elaborado, mas demonstra o

resultado contraintuitivo que os dados *consistentes* com uma hipótese são

não necessariamente *em favor* da hipótese.

Se este resultado ainda o incomoda, esta forma de pensar pode ajudar: o

os dados consistem num evento comum, tipo â??COPY15™ sangue, e um evento raro, tipo

10 euros de sangue 19. Se Oliver é o responsável pelo evento comum, isso deixa o

acontecimento raro e inexplicável. Se Oliver não conta com o sangue "COPY5" para o "˜Oâ", nós

têm duas hipóteses de encontrar alguém na população com â˜ABâ â™ de sangue. E

esse factor de dois faz a diferença.

**Exercício:** Suponha que, com base noutras provas, a sua crença anterior na culpa de Oliver é de 90%.  Quanto é que as provas de sangue nesta secção mudariam as suas crenças?  E se inicialmente pensasse que havia apenas 10% de probabilidades da sua culpa?

In [18]:
# Solution goes here

In [19]:
# Solution goes here

## Acrescenta

A segunda metade deste capítulo é sobre a distribuição de somas e resultados de outras operações.

Começaremos com um problema de avanço, onde nos são dadas as entradas e calculada a distribuição da saída.

Depois trabalharemos em problemas inversos, onde nos são dadas as saídas e calculamos a distribuição das entradas.

Como primeiro exemplo, suponha que lança dois dados e os adiciona. Qual é a distribuição da soma? 

Iâ™ll utiliza a seguinte função para criar um `Pmf` que representa o

possíveis resultados de um dado:

In [20]:
import numpy as np
from empiricaldist import Pmf

def make_die(sides):
    outcomes = np.arange(1, sides+1)
    die = Pmf(1/sides, outcomes)
    return die

Num dado de seis lados, os resultados são de 1 a 6, todos

igualmente provável.

In [21]:
die = make_die(6)

In [22]:
from utils import decorate

die.bar(alpha=0.4)
decorate(xlabel='Outcome',
         ylabel='PMF')

Se lançarmos dois dados e os somarmos, há 11 resultados possíveis, 2

até 12, mas não são igualmente prováveis. Para calcular a distribuição

da soma, temos de enumerar os resultados possíveis. 

E é assim que esta função funciona:

In [23]:
def add_dist(pmf1, pmf2):
    """Compute the distribution of a sum."""
    res = Pmf()
    for q1, p1 in pmf1.items():
        for q2, p2 in pmf2.items():
            q = q1 + q2
            p = p1 * p2
            res[q] = res(q) + p
    return res

Os parâmetros são objectos `Pmf` que representam distribuições.

Os loops iteram através das quantidades e probabilidades nos objectos `Pmf`.

Cada vez através do loop `q` recebe a soma de um par de quantidades, e `p` recebe a probabilidade do par.

Como a mesma soma pode aparecer mais de uma vez, temos de somar a probabilidade total para cada soma.

Repare num elemento subtil desta linha:

```
            res[q] = res(q) + p
```

Utilizo parênteses do lado direito da tarefa, que retorna 0 se `q` ainda não aparecer em `res`.

Utilizo parênteses no lado esquerdo da atribuição para criar ou actualizar um elemento em `res`; utilizar parênteses no lado esquerdo não funcionaria.

O `Pmf' fornece o `add_dist', que faz a mesma coisa.

Pode chamá-lo como um método, como este:

In [24]:
twice = die.add_dist(die)

Ou como uma função, como esta:

In [25]:
twice = Pmf.add_dist(die, die)

E o resultado é o seguinte:

In [26]:
from utils import decorate

def decorate_dice(title=''):
    decorate(xlabel='Outcome',
             ylabel='PMF',
             title=title)

In [27]:
twice = add_dist(die, die)
twice.bar(color='C1', alpha=0.5)
decorate_dice()

Se tivermos uma sequência de objectos `Pmf` que representam dados, podemos calcular a distribuição da soma desta forma:



In [28]:
def add_dist_seq(seq):
    """Compute Pmf of the sum of values from seq."""
    total = seq[0]
    for other in seq[1:]:
        total = total.add_dist(other)
    return total

A título de exemplo, podemos fazer uma lista de três dados como esta:

In [29]:
dice = [die] * 3

E podemos calcular a distribuição da sua soma desta forma.

In [30]:
thrice = add_dist_seq(dice)

A figura seguinte mostra como são estas três distribuições:

- A distribuição de um único dado é uniforme de 1 a 6.

- A soma de dois dados tem uma distribuição triangular entre 2 e 12.

- A soma de três dados tem uma distribuição em forma de sino entre 3

    e 18.

In [31]:
import matplotlib.pyplot as plt

die.plot(label='once')
twice.plot(label='twice', style='--')
thrice.plot(label='thrice', style=':')

plt.xticks([0,3,6,9,12,15,18])
decorate_dice(title='Distributions of sums')

Como um aparte, este exemplo demonstra o Teorema do Limite Central, que diz que a distribuição de uma soma converge para uma distribuição normal em forma de sino, pelo menos sob algumas condições.

## Sensibilidade ao glúten

Em 2015 li um artigo que testou se as pessoas diagnosticadas com sensibilidade ao glúten (mas não com doença celíaca) eram capazes de distinguir a farinha de glúten da farinha sem glúten num desafio cego

([you can read the paper here](https://onlinelibrary.wiley.com/doi/full/10.1111/apt.13372)).

Dos 35 sujeitos, 12 identificaram correctamente a farinha de glúten com base em

retomada dos sintomas enquanto o comiam. Outros 17 erradamente

identificaram a farinha sem glúten com base nos seus sintomas, e 6 foram

incapaz de distinguir.

Os autores concluem, "O desafio do glúten duplo-cego induz o sintoma

recidiva em apenas um terço dos pacientes".

Esta conclusão parece-me estranha, porque se nenhum dos pacientes fosse

sensíveis ao glúten, seria de esperar que alguns deles identificassem a farinha de glúten por acaso. 

Então eis a questão: com base nestes dados, quantos dos sujeitos são sensíveis ao glúten e quantos estão a adivinhar?

Podemos usar o Teorema de Bayes para responder a esta pergunta, mas primeiro temos de tomar algumas decisões de modelação. Vou assumir:

- As pessoas que são sensíveis ao glúten têm 95% de probabilidades de

    identificar a farinha de glúten nas condições de desafio, e

- As pessoas que não são sensíveis têm 40% de hipóteses de identificar o

    farinha de glúten por acaso (e uma probabilidade de 60% de escolher o

    outra farinha ou não conseguir distinguir).

Estes valores particulares são arbitrários, mas os resultados não são sensíveis a estas escolhas.

Vou resolver este problema em duas etapas. Primeiro, assumindo que sabemos quantos assuntos são sensíveis, vou calcular a distribuição dos dados. 

Depois, utilizando a probabilidade dos dados, calcularei a distribuição posterior do número de pacientes sensíveis.

O primeiro é o **problema em frente**; o segundo é o **inverso

problema**.

## O Problema do Avanço

Suponhamos que sabemos que 10 dos 35 sujeitos são sensíveis ao glúten. Que

significa que 25 não o são:

In [32]:
n = 35
num_sensitive = 10
num_insensitive = n - num_sensitive

Cada assunto sensível tem uma probabilidade de 95% de identificar a farinha de glúten,

por isso o número de identificações correctas segue uma distribuição binomial.

Vou utilizar `make_binomial`, que definimos em <<_TheBinomialDistribution>>, para fazer um `Pmf` que representa a distribuição binomial.

In [33]:
from utils import make_binomial

dist_sensitive = make_binomial(num_sensitive, 0.95)
dist_insensitive = make_binomial(num_insensitive, 0.40)

Os resultados são as distribuições do número de identificações correctas em cada grupo.

Agora podemos utilizar o `add_dist` para calcular a distribuição do número total de identificações correctas:

In [34]:
dist_total = Pmf.add_dist(dist_sensitive, dist_insensitive)

Aqui estão os resultados:

In [35]:
dist_sensitive.plot(label='sensitive', style=':')
dist_insensitive.plot(label='insensitive', style='--')
dist_total.plot(label='total')

decorate(xlabel='Number of correct identifications',
         ylabel='PMF',
         title='Gluten sensitivity')

Esperamos que a maioria dos sujeitos sensíveis identifique correctamente a farinha de glúten.

Dos 25 sujeitos insensíveis, esperamos cerca de 10 a identificar a farinha de glúten por acaso.

Por isso, esperamos cerca de 20 identificações correctas no total.

Esta é a resposta para o problema futuro: dado o número de assuntos sensíveis, podemos calcular a distribuição dos dados.

## O Problema Inverso

Agora vamos resolver o problema inverso: tendo em conta os dados, vamos calcular a distribuição posterior do número de sujeitos sensíveis.

Eis como.  Vou percorrer os possíveis valores de `num_sensitive` e calcular a distribuição dos dados para cada um:

In [36]:
import pandas as pd

table = pd.DataFrame()
for num_sensitive in range(0, n+1):
    num_insensitive = n - num_sensitive
    dist_sensitive = make_binomial(num_sensitive, 0.95)
    dist_insensitive = make_binomial(num_insensitive, 0.4)
    dist_total = Pmf.add_dist(dist_sensitive, dist_insensitive)    
    table[num_sensitive] = dist_total

O laço enumera os possíveis valores de `num_sensitive`.

Para cada valor, calcula a distribuição do número total de identificações correctas, e armazena o resultado como uma coluna num "DataFrame" de Pandas.

In [37]:
table.head(3)

A figura seguinte mostra colunas seleccionadas da "DataFrame", correspondendo a diferentes valores hipotéticos de `num_sensitive':

In [38]:
table[0].plot(label='num_sensitive = 0')
table[10].plot(label='num_sensitive = 10')
table[20].plot(label='num_sensitive = 20', style='--')
table[30].plot(label='num_sensitive = 30', style=':')
    
decorate(xlabel='Number of correct identifications',
         ylabel='PMF',
         title='Gluten sensitivity')

Agora podemos utilizar esta tabela para calcular a probabilidade dos dados:

In [39]:
likelihood1 = table.loc[12]

"loc" selecciona uma fila a partir do "DataFrame".

A linha com índice 12 contém a probabilidade de 12 identificações correctas para cada valor hipotético de `num_sensitive`.

E é exactamente essa a probabilidade de precisarmos de fazer uma actualização Bayesiana.

Vou utilizar um anterior uniforme, o que implica que ficaria igualmente surpreendido com qualquer valor de `num_sensitive`:

In [40]:
hypos = np.arange(n+1)
prior = Pmf(1, hypos)

E aqui está a actualização:



In [41]:
posterior1 = prior * likelihood1
posterior1.normalize()

Para comparação, também calculo o posterior para outro resultado possível, 20 identificações correctas.

In [42]:
likelihood2 = table.loc[20]
posterior2 = prior * likelihood2
posterior2.normalize()

A figura seguinte mostra distribuições posteriores de `num_sensitive` com base nos dados reais, 12 identificações correctas, e o outro resultado possível, 20 identificações correctas.

In [43]:
posterior1.plot(label='posterior with 12 correct', color='C4')
posterior2.plot(label='posterior with 20 correct', color='C1')

decorate(xlabel='Number of sensitive subjects',
         ylabel='PMF',
         title='Posterior distributions')

Com 12 identificações correctas, a conclusão mais provável é que nenhum dos sujeitos é sensível ao glúten.

Se tivesse havido 20 identificações correctas, a conclusão mais provável seria que 11-12 dos sujeitos eram sensíveis.

In [44]:
posterior1.max_prob()

In [45]:
posterior2.max_prob()

## Resumo

Este capítulo apresenta dois tópicos que quase não estão relacionados, excepto que tornam o título do capítulo cativante.

A primeira parte do capítulo é sobre a Regra de Bayes, evidência, e como podemos quantificar a força da evidência usando um rácio de probabilidade ou factor Bayes.

A segunda parte é sobre o `add_dist`, que calcula a distribuição de uma soma.

Podemos utilizar esta função para resolver problemas futuros e inversos; ou seja, dados os parâmetros de um sistema, podemos calcular a distribuição dos dados ou, dados os dados, podemos calcular a distribuição dos parâmetros.

No próximo capítulo, calcularemos as distribuições de mínimos e máximos, e utilizá-las-emos para resolver mais problemas Bayesianos.

Mas primeiro talvez queira trabalhar nestes exercícios.

## Exercícios

**Exercício:** Vamos usar a Regra de Bayes para resolver o problema de Elvis de <<_Distribuições>>>:

> Elvis Presley teve um irmão gémeo que morreu à nascença. Qual é a probabilidade de Elvis ter sido um gémeo idêntico?

Em 1935, cerca de 2/3 dos gémeos eram fraternais e 1/3 eram idênticos.

A pergunta contém duas peças de informação que podemos utilizar para actualizar esta anterior.

* Primeiro, o gémeo de Elvis também era masculino, o que é mais provável se fossem gémeos idênticos, com um rácio de probabilidade de 2.

* Além disso, o gémeo de Elvis morreu à nascença, o que é mais provável se fossem gémeos idênticos, com um rácio de probabilidade de 1,25.

Se estiver curioso sobre a origem desses números, escrevi [a blog post about it](https://www.allendowney.com/blog/2020/01/28/the-elvis-problem-revisited).

In [46]:
# Solution goes here

In [47]:
# Solution goes here

In [48]:
# Solution goes here

**Exercício:** O seguinte é um xx_markdown_link_link_xx, atribuído ao Facebook:

> Está prestes a embarcar num avião para Seattle. Quer saber se deve trazer um guarda-chuva. Telefona a 3 amigos seus que lá vivem ao acaso e pergunta a cada um independentemente se está a chover. Cada um dos seus amigos tem 2/3 de hipóteses de lhe dizer a verdade e 1/3 de se meter consigo ao mentir. Todos os 3 amigos lhe dizem que "Sim" está a chover. Qual é a probabilidade de estar realmente a chover em Seattle?

Use a Regra de Bayes para resolver este problema.  Como um anterior, pode assumir que chove em Seattle cerca de 10% do tempo.

Esta questão causa alguma confusão sobre as diferenças entre as interpretações Bayesianas e as interpretações de probabilidade dos frequentadores; se estiver curioso sobre este ponto, [I wrote a blog article about it](http://allendowney.blogspot.com/2016/09/bayess-theorem-is-not-optional.html).

In [49]:
# Solution goes here

In [50]:
# Solution goes here

In [51]:
# Solution goes here

**Exercício:** [According to the CDC](https://www.cdc.gov/tobacco/data_statistics/fact_sheets/health_effects/effects_cig_smoking), as pessoas que fumam são cerca de 25 vezes mais susceptíveis de desenvolver cancro do pulmão do que os não fumadores.

[Also according to the CDC](https://www.cdc.gov/tobacco/data_statistics/fact_sheets/adult_data/cig_smoking/index.htm), cerca de 14\% dos adultos nos E.U.A. são fumadores.

Se se souber que alguém tem cancro do pulmão, qual é a probabilidade de ser fumador?

In [52]:
# Solution goes here

In [53]:
# Solution goes here

In [54]:
# Solution goes here

**Exercício:** Em *Dungeons & Dragons*, a quantidade de danos que um duende pode suportar é a soma de dois dados de seis lados. A quantidade de dano que se inflige com uma espada curta é determinada pelo lançamento de um dado de seis lados.

Um duende é derrotado se o dano total que inflige for maior ou igual ao montante que pode suportar.

Suponha que está a combater um duende e que já infligiu 3 pontos de dano. Qual é a sua probabilidade de derrotar o goblin com o seu próximo ataque bem sucedido?

Dica: Pode utilizar `Pmf.add_dist` para adicionar uma quantidade constante, como 3, a um `Pmf` e `Pmf.sub_dist` para calcular a distribuição dos pontos restantes.

In [55]:
# Solution goes here

In [56]:
# Solution goes here

In [57]:
# Solution goes here

In [58]:
# Solution goes here

In [59]:
# Solution goes here

In [60]:
# Solution goes here

**Exercício:** Suponha que tenho uma caixa com um dado de 6 faces, um dado de 8 faces, e um dado de 12 faces.

Escolho um dos dados ao acaso, lanço-o duas vezes, multiplico os resultados, e informo que o produto é 12.

Qual é a probabilidade de eu ter escolhido o dado de 8 lados?

Dica: `Pmf` fornece uma função chamada `mul_dist` que pega em dois objectos `Pmf` e devolve um `Pmf` que representa a distribuição do produto.

In [61]:
# Solution goes here

In [62]:
# Solution goes here

In [63]:
# Solution goes here

In [64]:
# Solution goes here

**Exercício:** *Betrayal at House on the Hill* é um jogo de estratégia em que personagens com diferentes atributos exploram uma casa assombrada.  Dependendo dos seus atributos, as personagens lançam diferentes números de dados.  Por exemplo, se tentarem uma tarefa que depende do conhecimento, o Professor Longfellow lança 5 dados, a Madame Zostra lança 4, e o Ox Bellows lança 3.  Cada dado rende 0, 1, ou 2 com igual probabilidade.  

Se um personagem escolhido aleatoriamente tentar uma tarefa três vezes e lançar um total de 3 na primeira tentativa, 4 na segunda, e 5 na terceira, que personagem pensa que foi?

In [65]:
# Solution goes here

In [66]:
# Solution goes here

In [67]:
# Solution goes here

In [68]:
# Solution goes here

In [69]:
# Solution goes here

In [70]:
# Solution goes here

In [71]:
# Solution goes here

**Exercício:** Há 538 membros do Congresso dos Estados Unidos. Suponhamos que auditamos as suas carteiras de investimento e verificamos que 312 deles têm um desempenho superior ao do mercado.

Vamos supor que um membro honesto do Congresso tem apenas 50% de hipóteses de superar o mercado, mas um membro desonesto que negoceie com informação privilegiada tem 90% de hipóteses.  Quantos membros do Congresso são honestos?

In [72]:
# Solution goes here

In [73]:
# Solution goes here

In [74]:
# Solution goes here

In [75]:
# Solution goes here

In [76]:
# Solution goes here

In [77]:
# Solution goes here

In [78]:
# Solution goes here